In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters     # This function specifies the (x, y) coordinates  
    y1 = image.shape[0]                    # of the averaged slope and intercept array returned 
    y2 = int(y1 * (3/5))                   # by the average_slope_intercept function
    x2 = int((y2 - intercept)/slope)
    x1 = int((y1 - intercept)/slope)
    return np.array([x1, y1, x2, y2])


def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:                                  # This function takes in the (x, y) coordinates of the lane lines detected and transforms them 
        x1, y1, x2, y2 = line.reshape(4)                # into a (slope, y-intercept) tuple which it passes to a make_coordinates function and then finally 
        parameters = np.polyfit((x1, x2), (y1, y2), 1)  # returns a single averaged (x, y) coordinate for both the left and right lane lines
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0: 
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_averaged = np.average(left_fit, axis=0)
    right_fit_averaged = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_averaged)
    right_line = make_coordinates(image, right_fit_averaged)
    return np.array([left_line, right_line])


def canny_edge(image):                                   
    lane_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)    # This function applies the canny edge detection function to our 
    lane_blur = cv.GaussianBlur(lane_gray, (5, 5), 0)    # camera images to detect lines, including lane lines, in our image.
    lane_canny = cv.Canny(lane_blur, 50, 150)
    return lane_canny


def display_lines(image, lines):
    line_image = np.zeros_like(image)    # This function draws our detected lane lines onto a black 
    if lines is not None:                # image of the same size as our camera image
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 
                    thickness= 10)
    return line_image


def roi(image):
    height = image.shape[0]                         # This function masks the region containing our lane lines
    polygons = np.array([                           # in our canny image and isoltes it making the detected lane 
        [(200, height), (1100, height), (550, 250)] # lines the only lines in our image.
                        ])
    mask = np.zeros_like(image)
    cv.fillPoly(mask, polygons, (255, 255, 255))
    masked_image = cv.bitwise_and(image, mask)
    return masked_image


capt = cv.VideoCapture("test2.mp4")
while capt.isOpened():
    _, frame_img = capt.read()
    canny_image = canny_edge(frame_img)         # calls the canny function on our camera image to return an image with all edges detected and displayed

    cropped_image = roi(canny_image)       # calls the roi function on our canny image which returns an image in which the lane lines have been isolated

    lines_detected = cv.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), # We use the Hough transform technique to detect straight lines 
                        minLineLength=40, maxLineGap=10)                          # in our region of interest thus identifying our lane lines

    averaged_lines = average_slope_intercept(frame_img, lines_detected)  # This calls the average_slope_intercept function on the detected lane lines and the camera 
                                                                    # image which returns the averaged lane lines

    line_image_ret = display_lines(frame_img, averaged_lines)  # This function is passed (2) arguments and it returns a black image with our lane lines drawn on it

    combo_image = cv.addWeighted(frame_img, 0.8, line_image_ret, 1, 1) # This function combines our actual camera image with our lane line image and gives our 
                                                                # camera image with linses drawn indictating our lane lines
    cv.imshow("Result", combo_image)
    if cv.waitKey(1) == ord("m"):
        break
capt.release() 
cv.destroyAllWindows()

